<a href="https://colab.research.google.com/github/a01685424/CloudComputing_TC4031/blob/main/Equipo19_semana05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Procesamiento de Lenguaje Natural (NLP)**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad Semanas 05**

###**5.2 Actividad: Continuando con caso de estudio Amazon-Yelp-Imdb**

#**Matricula-Alumnos:** 

**A01793547.- Eddie Guadalupe Elorza Ruiz**

**A01685424.-Fernando Alfredo Rojas Estrella**

**A00174646.-José Ramiro Adán Charles**

**A01793704.-Marco Antonio Vázquez Morales**

**A01793131.- Rubén Marcos Ramos Guerrero**

En la actividad de esta semana trabajarás en equipos con el modelo de vectores continuos/embebidos FastText,
es decir, el modelo desarrollado por Facebook en 2016.
Una manera de trabajar con estos modelos pre-entrenados, es generando el vocabulario a partir de tu conjunto
de datos de entrenamiento. Para cada palabra de tu vocabulario, podrás sustituirlo por su correspondiente
vector continuo. En caso de que no exista el vector para una palabra en particular, se puede eliminar dicha
palabra, o bien sustituirla por el vector continuo más cercano. En esta actividad deberás aplicar esta segunda
opción. Existen diversas propuestas para utilizar dichos vectores continuos como entrada para modelos de
aprendizaje automático. En particular, en esta actividad cada enunciado será sustituido por el vector promedio
de todos los tokens que lo forman.

# Paso 1

Descarga los 3 archivos de Canvas. En particular, el archivo de datos de IMDb ya no requiere
transformarse para obtener sus 1000 registros. Al cargar los datos de los tres archivos deberás
tener un DataFrame de Pandas de 3000 registros, con sus etiquetas. Los archivos los encuentras en
Canvas y se llaman: amazon5.txt, imdb5.txt, yelp5.txt.

In [19]:
# Importando liberías

import pandas as pd  
import numpy as np

import nltk
from nltk.corpus import stopwords

import re
import string

from nltk.stem import WordNetLemmatizer     # Para implementar lematización

In [20]:
nltk.download('stopwords')    # Descarga de stopwords
nltk.download('wordnet')      # Descarga para lematización

#nltk.download('punkt')        # Descarga tokenizador que ayuda a dividr el texto en enunciados mediante un modelo no-supervisado.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Carga de los archivos

dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', delimiter='\t', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')


print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)


In [4]:
# Revisando sets
print('\n- - - - - - - - - - - - - - - - - - - - - - -\ndfa')
dfa.info()
print('\n- - - - - - - - - - - - - - - - - - - - - - -\ndfi')
dfi.info()
print('\n- - - - - - - - - - - - - - - - - - - - - - -\ndfy')
dfy.info()


- - - - - - - - - - - - - - - - - - - - - - -
dfa
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB

- - - - - - - - - - - - - - - - - - - - - - -
dfi
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  1000 non-null   object 
 1   label   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 15.8+ KB

- - - - - - - - - - - - - - - - - - - - - - -
dfy
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null  

In [5]:
# dfi tienen un problema ya que la columna label está vacía
dfii= dfi.copy()
i= 0
for ind, fila in dfii.iterrows():
  string= dfii.iloc[i,0]
  label= re.findall(r'(\d)$', string)
  dfii.iloc[i,1]= int(label[0])
  #print(dfii.iloc[1,1])
  i+=1
# Se obtienen los valores de review de cada fila en label
dfii['label'].value_counts()   # Recordemos que debemos tener un 50% de comentarios positivos y de negativos.

0.0    500
1.0    500
Name: label, dtype: int64

In [6]:
# Se concatenan los tres conjuntos

df = pd.concat([dfa, dfii, dfy], ignore_index=True)   # Se utiliza el "ignore_index", para no reinciar cada vez los índices

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  3000 non-null   object 
 1   label   3000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 47.0+ KB


In [7]:
df['label'].value_counts()   # Recordemos que debemos tener un 50% de comentarios positivos y de negativos.

0.0    1500
1.0    1500
Name: label, dtype: int64

In [8]:
df.head()

,review,label
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,Tied to charger for conversations lasting more...,0.0
4,The mic is great.,1.0


# Paso 2

Realiza de nuevo un proceso de limpieza. Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización. Como aplicaremos modelos embebidos pre-entrenados, queremos palabras lo más cercanas a las existentes en un idioma, inglés en este caso. Aplica y justifica cualquier otro proceso de limpieza que consideres adecuado. Recuerda que en esta actividad se usarán vectores embebidos para un problema de clasificación, por lo que deberás tomar de acuerdo a este contexto. Justifica todas las transformaciones que se apliquen.

In [9]:
# Verificamos la existencia de los comentarios númericos que habíamos encontrado en tareas anteriores

print(df.iloc[1125,:])
print(df.iloc[1788,:])

review    10/10    1
label            1.0
Name: 1125, dtype: object
review    10/10    1
label            1.0
Name: 1788, dtype: object


In [10]:
# Se realiza la adecuación del texto

df.iloc[1125,0]= 'Excellent movie.'
df.iloc[1788,0]= 'Excellent movie.'

print(df.iloc[1125,:])
print(df.iloc[1788,:])

review    Excellent movie.
label                  1.0
Name: 1125, dtype: object
review    Excellent movie.
label                  1.0
Name: 1788, dtype: object


In [21]:
def clean_tok(doc):
  ##############################################################################
  # 1 Simplificar todo a minúsculas
  cadena= re.sub(r"'", "", doc.lower())             # Aseguramos que las negaciones en palabras no se borren.
  #print('cadena= ', cadena)
  # 2 Considerando solamente tokens de longitud mayor a 1
  cadena= re.sub(r'(\W\w{1}\W)', ' ', cadena)
  #print('CADENA=', cadena)
  # 3 Solo considerar caracteres alfabéticos
  cadena= re.sub(r'[^a-zA-Z]', ' ', cadena)         # Consideramos caracteres letras solamente.
  cadena= re.sub(r'\s{2,}', ' ', cadena.strip())
  # 4 Tokenizado por palabra
  lista= re.split(r'\s', cadena)
  # 5 Eliminando stopwords
  negwords= [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
  mystopwords= [word for word in stopwords.words('english') if word not in negwords]
  tokenes= [word for word in lista if word not in mystopwords]
  # Aplicando lematización
  wnl = WordNetLemmatizer()
  tokens= [ wnl.lemmatize(token) for token in tokenes]
  # CODIGO
  ##############################################################################

  return tokens

In [22]:
# Separando la información para realizar su limpieza
 #     La "X" Datos de comentarios.
 #     La "Y" Variable de la evaluación.

X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s

assert X.shape == (3000,)           # verificando que tenemos la dimensiones esperadas.
assert Y.shape == (3000,)

In [28]:
# Aplicamos el proceso de limpieza y tokenización:
print(X[0:5])
Xcleantok = [clean_tok(x) for x in X]

0    So there is no way for me to plug it in here i...
1                          Good case, Excellent value.
2                               Great for the jawbone.
3    Tied to charger for conversations lasting more...
4                                    The mic is great.
Name: review, dtype: object


In [26]:
# Primer ejercicio de limpieza

for x in Xcleantok[0:5]:
  print(x)

['no', 'way', 'plug', 'u', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversation', 'lasting', 'minute', 'major', 'problem']
['mic', 'great']


# Paso 3

Llamar Xclean a los comentarios procesados y Y a las etiquetas. Realicemos una partición aleatoria
con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba,
respectivamente. Verifica que obtienes 2100 registros de entrenamiento y 450 para cada uno de
validación y prueba.

In [36]:
# Partición de la información
from sklearn.model_selection import train_test_split

Xclean= Xcleantok.copy()
Y

Xtrain, Xtest_Xval, Ytrain, Ytest_Yval= train_test_split(Xclean, Y, test_size=0.3, random_state=42)
Xval, Xtest, Yval, Ytest= train_test_split(Xtest_Xval, Ytest_Yval, test_size=0.5, random_state=42)

print('Tamaño entrenamiento=', len(Xtrain))
print('Tamaño validación=', len(XVal))
print('Tamaño prueba=', len(Xtest))

Tamaño entrenamiento= 2100
Tamaño validación= 450
Tamaño prueba= 450


# Paso 4

Usando el conjunto de entrenamiento genera un vocabulario que no sea mayor a 1500 palabras, ni
menor a 1000. ¿Por qué es importante acotar un vocabulario inferior y superiormente? ¿Por qué
debe usarse solamente el conjunto de entrenamiento para generar el diccionario? Con este
vocabulario que obtienes, filtra los conjuntos de entrenamiento, validación y prueba, de esta
manera todos los comentarios usarán solamente palabras válidas de acuerdo a este vocabulario.
Indica el tamaño del vocabulario obtenido.
Hasta este punto básicamente has realizado transformaciones muy análogas a las de la semana
pasada y que son válidas para muchos de los procesos dentro del análisis de textos. En dado caso
comenta con tus compañeros de equipo qué diferencias has observado. Veamos ahora la diferencia
con respecto a las matrices Tf-idf que aplicaste la semana pasada, con respecto a los vectores preentrenados
embebidos.

In [55]:
# Generando vocabulario de entrenamiento

from collections import Counter

frec= Counter()
for listas in Xtrain:
  frec.update(listas)

frecuencia= frec.most_common()
vocabulario= list(frec.keys())
#print('Vocabulario= ', Vocabulario)
print('Tamaño vocabulario de entrenamiento= ', len(vocabulario))

# Se quedan las palabras con mayor frecuencia
vocatrain= []
for palabra, frec in frecuencia:
  if len(vocatrain) < 1490:
    if frec > 1:
      vocatrain.append(palabra)
    #print(f'{palabra}: {frec}')

print('\nTamaño vocabulario acortado= ', len(vocatrain))


Tamaño vocabulario de entrenamiento=  3722

Tamaño vocabulario acortado=  1490


## Preguntas

- ¿Por qué es importante acotar un vocabulario inferior y superiormente?
  - **Respuesta:** Una de las principales razones es para evitar el sobre ajuste de los datos, para evitar palabras tan poco comunes o frecuentes que hagan que el modelo se comporte sobre entrenado. Acotar el vocabulario también ayudará a generalizar el modelo representando de mejor manera las características generales de los comentarios que se están analizando.

- ¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el diccionario?
  - **Respuesta:** Una de las razones principales es por qué al limitar el entrenamiento del modelo al conjunto de entrenamiento se efita la filtración de información, permitiendo la obtención de escenarios más parecidos a la realidad a la que se enfrentará el modelo cuando logre generalizar de forma adecuada.

# Paso 5

Utilizarás los vectores embebidos FastText preentrenados por Facebook.
a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de
Google y Glove de Stanford. Puedes consultar sus páginas correspondientes:
https://fasttext.cc/
https://code.google.com/archive/p/word2vec/
https://nlp.stanford.edu/projects/glove/

# Paso 6

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar
un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el
“valor” será su vector embebido de dimensión 300. Este diccionario deberá ser del mismo tamaño
que el vocabulario previo que hayas construido previamente.
https://fasttext.cc/docs/en/crawl-vectors.html
NOTA: Debido a la cantidad de recursos computacionales que demanda cargar los vectores
FastText (son 2 millones de vectores), es recomendable que una vez que generes el nuevo
vocabulario de vectores embebidos, guardes dicho diccionario en un archivo (pickle, npz o el que
consideres más adecuado). Una vez realizado lo anterior, puedes borrar la variable de FastText
para liberar memoria RAM. De esta manera, ya tienes tu vocabulario de vectores embebidos de
acuerdo a los tokens que consideras más adecuados para tu problema y puedes usarlo rápidamente
cuando lo necesites. En dado caso apóyense entre los miembros del equipo de tener dificultades
para generar el vocabulario y por mientras puedes usar el archivo del vocabulario que alguno haya
generado

# Paso 7

Una manera de utilizar los vectores embebidos con modelos de aprendizaje automático en
documentos de texto, es asignar a cada comentario filtrado el vector embebido de dimensión 300
que resulta de promediar todos sus tokens. Así, en este ejercicio deberás generar los arreglos
correspondientes para los conjuntos de entrenamiento, validación y prueba. Los llamaremos
trainEmb, valEmb y testEmb, respectivamente. ¿Cuáles son sus dimensiones? ¿Se podrían usar para
su representación matrices dispersas (sparse matrices) como en el caso de la matriz Tf-idf?
Responde a dichas preguntas.

# Paso 8

Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños (accuracy). Compara los resultados con los de la semana anterior.

# Paso 9

Obtener la matriz de confusión e interpretar sus valores.

# Paso 10

Comenta con tus compañeros de equipo los pasos realizados en esta actividad e incluyan sus
conclusiones finales.